In [101]:
import nltk
from nltk.corpus import wordnet as wn
import json
import polars as pl

# Download WordNet data
nltk.download('wordnet')

import sys
sys.path.append('..')
from utils import utils as u

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\riccardo.gino\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [102]:
def get_genus_differentia(definition: str) -> (list, list):
    genus, differentia = [], []
    # split definition for , or .

    comma_split = definition.split(',')

    splitted_sent = []

    for cs in comma_split:
        if cs.split('.') != '':
            splitted_sent += cs.split('.')

    if '' in splitted_sent:
        splitted_sent.remove('')

    # print(splitted_sent)

    all_occurrences = [
        "it", "is", "'s", "a", "an", "the",
        "that", "of", "for"
    ]

    targets = {
        1: [["it"], ["is", "'s"], ["a", "an", "the", "for"]],
        2: [["that", "of"]],
        3: [["a", "an", "for"]]
    }

    for sentence in splitted_sent:
        tokenized_def = u.get_lemmatized_tokens_list_pos(u.tokenizer(sentence))
        print(tokenized_def)

        checked_tokens = []

        for i in range(0, len(tokenized_def)):
            if tokenized_def[i][0] not in checked_tokens:
                condition = tokenized_def[i][0].lower() in all_occurrences

                checked_tokens.append(tokenized_def[i][0])

                if tokenized_def[i][0].lower() == targets[2][0][0] or (i == 1 and tokenized_def[i][0].lower() == targets[2][0][1]):
                    if tokenized_def[i-1][0] not in genus:
                        genus.append(tokenized_def[i-1][0])

                elif (i < len(tokenized_def)-1):
                    if tokenized_def[i][0].lower() == targets[3][0][0] or tokenized_def[i][0].lower() == targets[3][0][1] or tokenized_def[i][0].lower() == targets[3][0][2]:
                        if tokenized_def[i+1][0] not in genus and tokenized_def[i+1][1] == 'n':
                            genus.append(tokenized_def[i+1][0])
                            checked_tokens.append(tokenized_def[i+1][0])
                    elif tokenized_def[i][1] == 'a' and tokenized_def[i+1][1] == 'n':
                        genus.append(tokenized_def[i+1][0])
                        if not condition:
                            differentia.append(tokenized_def[i][0])
                        checked_tokens.append(tokenized_def[i+1][0])
                    else:
                        if not condition:
                            differentia.append(tokenized_def[i][0])
                elif (i < len(tokenized_def)-3):
                    if tokenized_def[i][0].lower() == targets[1][0][0] and \
                        (tokenized_def[i+1][0] == targets[1][1][0] or tokenized_def[i+1][0] == targets[1][1][1]) and \
                            (tokenized_def[i+2][0] == targets[1][2][0] or tokenized_def[i+2][0] == targets[1][2][1] or tokenized_def[i+2][0] == targets[1][2][2]):
                        if tokenized_def[i+3][0] not in genus:
                            genus.append(tokenized_def[i+3][0])
                            checked_tokens.append(tokenized_def[i+1][0])
                            checked_tokens.append(tokenized_def[i+2][0])
                            checked_tokens.append(tokenized_def[i+3][0])
                    else:
                        if not condition:
                            differentia.append(tokenized_def[i][0])
                else:
                    if not condition:
                        differentia.append(tokenized_def[i][0])
    print(genus, differentia)
    return genus, differentia


In [103]:
get_genus_differentia("Usable for access from one area to another")

[('Usable', 'a'), ('for', 'unk'), ('access', 'n'), ('from', 'unk'), ('one', 'unk'), ('area', 'n'), ('to', 'unk'), ('another', 'unk')]
['access'] ['Usable', 'from', 'one', 'area', 'to', 'another']


(['access'], ['Usable', 'from', 'one', 'area', 'to', 'another'])

In [104]:
# read csv with polars module
df = json.loads(pl.read_csv(
    "../datasets/TLN-definitions-23.tsv", separator='\t').write_json())

data = {}

for col in df['columns']:
    if col['name'] != '1':
        data[col['name']] = {}
        count = 1
        for v in col['values']:
            data[col['name']][count] = {}
            data[col['name']][count]['genus'], data[col['name']
                                                    ][count]['differentia'] = get_genus_differentia(v)
            count += 1


[('A', 'unk'), ('construction', 'n'), ('use', 'v'), ('to', 'unk'), ('divide', 'v'), ('two', 'unk'), ('room', 'n')]
[('temporarily', 'r'), ('close', 'v'), ('the', 'unk'), ('passage', 'n'), ('between', 'unk'), ('them', 'unk')]
['construction'] ['use', 'to', 'divide', 'two', 'room', 'temporarily', 'close', 'passage', 'between', 'them']
[('It', 'unk'), ("'s", 'v'), ('an', 'unk'), ('opening', 'n')]
[('it', 'unk'), ('can', 'unk'), ('be', 'v'), ('open', 'v'), ('or', 'unk'), ('close', 'v')]
['opening'] ['can', 'be', 'open', 'or', 'close']
[('An', 'unk'), ('object', 'n'), ('that', 'unk'), ('divide', 'v'), ('two', 'unk'), ('room', 'n')]
[('close', 'v'), ('an', 'unk'), ('hole', 'n'), ('in', 'unk'), ('a', 'unk'), ('wall', 'n')]
[('You', 'unk'), ('can', 'unk'), ('open', 'v'), ('the', 'unk'), ('door', 'n'), ('to', 'unk'), ('let', 'v'), ('people', 'n'), ('enter', 'v'), ('or', 'unk'), ('get', 'v'), ('out', 'r')]
['object', 'hole', 'wall'] ['divide', 'two', 'room', 'close', 'in', 'You', 'can', 'open', 

In [105]:
data

{'door': {1: {'genus': ['construction'],
   'differentia': ['use',
    'to',
    'divide',
    'two',
    'room',
    'temporarily',
    'close',
    'passage',
    'between',
    'them']},
  2: {'genus': ['opening'],
   'differentia': ['can', 'be', 'open', 'or', 'close']},
  3: {'genus': ['object', 'hole', 'wall'],
   'differentia': ['divide',
    'two',
    'room',
    'close',
    'in',
    'You',
    'can',
    'open',
    'door',
    'to',
    'let',
    'people',
    'enter',
    'or',
    'get',
    'out']},
  4: {'genus': ['access'],
   'differentia': ['Usable', 'from', 'one', 'area', 'to', 'another']},
  5: {'genus': ['Structure', 'area'],
   'differentia': ['Structure', 'delimit', 'and', 'allow', 'access', 'to']},
  6: {'genus': ['object'],
   'differentia': ['be',
    'use',
    'to',
    'block',
    'passage',
    'but',
    'can',
    'move',
    'pass']},
  7: {'genus': ['object', 'material', 'building', 'handle', 'body', 'lock'],
   'differentia': ['assembled',
    'his

In [106]:
def count_occurrences(lst):
    occurrences = {}
    
    for item in lst:
        if item in occurrences:
            occurrences[item] += 1
        else:
            occurrences[item] = 1
    
    return dict(sorted(occurrences.items(), key=lambda item: item[1], reverse=True))


In [107]:
result = {}
for word in data:
    result[word] = {}

    genus = []
    differentia = []

    for def_id in data[word]:
        genus += u.remove_punctuation_pos(
            u.remove_stop_words_pos(u.get_lemmatized_tokens_list_pos(data[word][def_id]['genus'])))
        differentia += u.remove_punctuation_pos(
            u.remove_stop_words_pos(u.get_lemmatized_tokens_list_pos(data[word][def_id]['differentia'])))
   

    # Calcola le occorrenze per 'genus' e 'differentia'
    genus_occurrences = count_occurrences(genus)
    differentia_occurrences = count_occurrences(differentia)

    # Filtra le voci con frequenza inferiore a 2
    filtered_genus_occurrences = {key: value for key, value in genus_occurrences.items() if value >= 1 and key[1] == 'n'}
    filtered_differentia_occurrences = {key: value for key, value in differentia_occurrences.items() if value >= 2 and key[1] == 'n'}

    result[word]['genus'] = filtered_genus_occurrences
    result[word]['differentia'] = filtered_differentia_occurrences



In [108]:
result

{'door': {'genus': {('room', 'n'): 7,
   ('wall', 'n'): 4,
   ('space', 'n'): 4,
   ('object', 'n'): 3,
   ('opening', 'n'): 2,
   ('area', 'n'): 2,
   ('building', 'n'): 2,
   ('body', 'n'): 2,
   ('mechanism', 'n'): 2,
   ('place', 'n'): 2,
   ('construction', 'n'): 1,
   ('hole', 'n'): 1,
   ('access', 'n'): 1,
   ('Structure', 'n'): 1,
   ('material', 'n'): 1,
   ('handle', 'n'): 1,
   ('lock', 'n'): 1,
   ('something', 'n'): 1,
   ('entrance', 'n'): 1,
   ('weather', 'n'): 1,
   ('chamber', 'n'): 1,
   ('barrier', 'n'): 1,
   ('hinge', 'n'): 1,
   ('Element', 'n'): 1,
   ('architecture', 'n'): 1,
   ('person', 'n'): 1,
   ('Object', 'n'): 1,
   ('house', 'n'): 1,
   ('point', 'n'): 1,
   ('structure', 'n'): 1},
  'differentia': {('access', 'n'): 10,
   ('room', 'n'): 8,
   ('use', 'n'): 6,
   ('passage', 'n'): 4,
   ('people', 'n'): 3,
   ('lock', 'n'): 3,
   ('space', 'n'): 3,
   ('wood', 'n'): 2,
   ('object', 'n'): 2,
   ('something', 'n'): 2,
   ('block', 'n'): 2,
   ('allow',

In [109]:
# vedo tra le definizioni dei synset degli iponimi di ogni genus e prendo i 5 con più parole in comune con la differentia

In [110]:
# prendo gli imponimi di ogni genus
hyponyms = {}
for word in result:
    hyponyms[word] = {}
    for genus in result[word]['genus']:
        hyponyms[word][genus] = {}
        for syns in wn.synsets(genus[0], genus[1]):
            for hyponym in syns.hyponyms():
                 hyponyms[word][genus][hyponym] = set(u.noise_reduction_en_pos(hyponym.definition()))

In [111]:
hyponyms

{'door': {('room',
   'n'): {Synset('anechoic_chamber.n.01'): {('chamber', 'n'),
    ('little', 'a'),
    ('reverberation', 'n')}, Synset('anteroom.n.01'): {('area', 'n'),
    ('entrance', 'n'),
    ('large', 'a'),
    ('reception', 'n'),
    ('room', 'n')}, Synset('back_room.n.01'): {('accessible', 'v'),
    ('establishment', 'n'),
    ('group', 'n'),
    ('locate', 'v'),
    ('privilege', 'v'),
    ('rear', 'n'),
    ('room', 'n'),
    ('usually', 'r')}, Synset('ballroom.n.01'): {('dance', 'v'),
    ('large', 'a'),
    ('mainly', 'r'),
    ('room', 'n'),
    ('use', 'v')}, Synset('barroom.n.01'): {('alcoholic', 'a'),
    ('counter', 'n'),
    ('drink', 'n'),
    ('establishment', 'n'),
    ('room', 'n'),
    ('serve', 'v')}, Synset('bathroom.n.01'): {('bathtub', 'n'),
    ('contain', 'v'),
    ('residence', 'n'),
    ('room', 'n'),
    ('shower', 'a'),
    ('toilet', 'n'),
    ('usually', 'r'),
    ('washbasin', 'n')}, Synset('bedroom.n.01'): {('primarily', 'r'),
    ('room', 'n'),
 

In [112]:
# per ogni definizione di hypomyms prendo quella con più parole in comune con la differentia
best_syns_list = {}
for word in hyponyms:
    max_score = 0
    best_syns = []
    for genus in hyponyms[word]:
        for syns in hyponyms[word][genus]:
            score = 0
            for tagged_token in hyponyms[word][genus][syns]:

                if tagged_token in result[word]['differentia']:
                    score +=1

            score = score/len(hyponyms[word][genus][syns])
            
            if score >= max_score:
                max_score = score
                best_syns.append((syns, score))
    best_syns_list[word] = []
    best_syns = sorted_list = sorted(best_syns, key=lambda x: x[1])
    i=0
    for s in best_syns:
        if s[1] == max_score or i < 5:
            best_syns_list[word].append(s[0])
            i+=1
    



In [113]:
best_syns_list

{'door': [Synset('anechoic_chamber.n.01'),
  Synset('anteroom.n.01'),
  Synset('toilet.n.01'),
  Synset('workroom.n.01'),
  Synset('cavity.n.02'),
  Synset('possible.n.01')],
 'ladybug': [Synset('birthplace.n.01'),
  Synset('end.n.10'),
  Synset('grave.n.02'),
  Synset('hiding_place.n.01'),
  Synset('high.n.05'),
  Synset('ephemerid.n.01')],
 'pain': [Synset('masking.n.02'),
  Synset('sound.n.02'),
  Synset('inconvenience.n.01'),
  Synset('owner.n.02')],
 'blurriness': [Synset('auditory_image.n.01'),
  Synset('imagination_image.n.01'),
  Synset('memory_image.n.01'),
  Synset('oculus_dexter.n.01'),
  Synset('oculus_sinister.n.01'),
  Synset('binocular_vision.n.01')]}

In [114]:
hyponyms

{'door': {('room',
   'n'): {Synset('anechoic_chamber.n.01'): {('chamber', 'n'),
    ('little', 'a'),
    ('reverberation', 'n')}, Synset('anteroom.n.01'): {('area', 'n'),
    ('entrance', 'n'),
    ('large', 'a'),
    ('reception', 'n'),
    ('room', 'n')}, Synset('back_room.n.01'): {('accessible', 'v'),
    ('establishment', 'n'),
    ('group', 'n'),
    ('locate', 'v'),
    ('privilege', 'v'),
    ('rear', 'n'),
    ('room', 'n'),
    ('usually', 'r')}, Synset('ballroom.n.01'): {('dance', 'v'),
    ('large', 'a'),
    ('mainly', 'r'),
    ('room', 'n'),
    ('use', 'v')}, Synset('barroom.n.01'): {('alcoholic', 'a'),
    ('counter', 'n'),
    ('drink', 'n'),
    ('establishment', 'n'),
    ('room', 'n'),
    ('serve', 'v')}, Synset('bathroom.n.01'): {('bathtub', 'n'),
    ('contain', 'v'),
    ('residence', 'n'),
    ('room', 'n'),
    ('shower', 'a'),
    ('toilet', 'n'),
    ('usually', 'r'),
    ('washbasin', 'n')}, Synset('bedroom.n.01'): {('primarily', 'r'),
    ('room', 'n'),
 

In [115]:
result

{'door': {'genus': {('room', 'n'): 7,
   ('wall', 'n'): 4,
   ('space', 'n'): 4,
   ('object', 'n'): 3,
   ('opening', 'n'): 2,
   ('area', 'n'): 2,
   ('building', 'n'): 2,
   ('body', 'n'): 2,
   ('mechanism', 'n'): 2,
   ('place', 'n'): 2,
   ('construction', 'n'): 1,
   ('hole', 'n'): 1,
   ('access', 'n'): 1,
   ('Structure', 'n'): 1,
   ('material', 'n'): 1,
   ('handle', 'n'): 1,
   ('lock', 'n'): 1,
   ('something', 'n'): 1,
   ('entrance', 'n'): 1,
   ('weather', 'n'): 1,
   ('chamber', 'n'): 1,
   ('barrier', 'n'): 1,
   ('hinge', 'n'): 1,
   ('Element', 'n'): 1,
   ('architecture', 'n'): 1,
   ('person', 'n'): 1,
   ('Object', 'n'): 1,
   ('house', 'n'): 1,
   ('point', 'n'): 1,
   ('structure', 'n'): 1},
  'differentia': {('access', 'n'): 10,
   ('room', 'n'): 8,
   ('use', 'n'): 6,
   ('passage', 'n'): 4,
   ('people', 'n'): 3,
   ('lock', 'n'): 3,
   ('space', 'n'): 3,
   ('wood', 'n'): 2,
   ('object', 'n'): 2,
   ('something', 'n'): 2,
   ('block', 'n'): 2,
   ('allow',